In [1]:
import os
import torch
import pandas as pd
import json
import re
import random
import numpy as np

from datasets import Dataset, load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer, SFTConfig
from huggingface_hub import login
from dotenv import load_dotenv

# ==============================================================================
# CONFIGURATION
# ==============================================================================

# Load environment variables
load_dotenv()
print("Environment variables loaded.")

# Hugging Face login
try:
    hf_token = os.getenv("HF_TOKEN")
    if hf_token:
        login(token=hf_token)
        print("Successfully logged into Hugging Face.")
    else:
        print("Hugging Face token not found.")
except Exception as e:
    print(f"Could not log into Hugging Face: {e}")

# Cache directory
HF_CACHE_DIR = "/cs/student/projects3/aisd/2024/ghanda/cache"
os.environ["HF_HOME"] = HF_CACHE_DIR
os.makedirs(HF_CACHE_DIR, exist_ok=True)
print(f"Hugging Face cache directory set to: {HF_CACHE_DIR}")

# Model configuration
MODEL_NAME = "google/gemma-2-2b"
FINE_TUNED_MODEL_BASE_NAME = "gemma-2b-personality-peft"

# QLoRA parameters
lora_r = 64
lora_alpha = 16
lora_dropout = 0.1

# Quantization parameters
use_4bit = True
bnb_4bit_compute_dtype = "bfloat16"
bnb_4bit_quant_type = "nf4"
use_nested_quant = False

# Training parameters
OUTPUT_DIR_BASE = "peft_output_models"
num_train_epochs = 2
fp16 = False
bf16 = True
per_device_train_batch_size = 1
gradient_accumulation_steps = 8
gradient_checkpointing = True
max_grad_norm = 0.3
learning_rate = 2e-4
weight_decay = 0.001
optim = "paged_adamw_8bit"
lr_scheduler_type = "cosine"
max_steps = -1
warmup_ratio = 0.03
group_by_length = True
save_steps = 0
logging_steps = 5
max_seq_length = 512
packing = False

# Device mapping
device_map = {"": 0}

# Dataset paths
PERSONALITY_DATASET_NAME = "holistic-ai/personality_manipulation"
BBQ_DATA_FILE = "/cs/student/projects3/aisd/2024/ghanda/bbq_ambiguous_with_metadata.csv"

# ==============================================================================
# HELPER FUNCTIONS
# ==============================================================================

def create_training_prompt_format(sample):
    """Format sample for training."""
    sample["text"] = f"<s>[INST] {sample['Question']} [/INST]{sample['Answer']}</s>"
    return sample

def create_inference_prompt_format(question, tokenizer):
    """Format question for inference."""
    return tokenizer.apply_chat_template(
        [{"role": "user", "content": question}],
        tokenize=False,
        add_special_tokens=True
    )

def extract_question_and_choices(full_prompt_string):
    """Extract question and choices from OpinionQA prompt."""
    question_match = re.search(r'<question>(.*?)</question>', full_prompt_string, re.DOTALL)
    choices_match = re.search(r'<choices>(.*?)</choices>', full_prompt_string, re.DOTALL)
    question_text = question_match.group(1).strip() if question_match else ""
    choices_raw_str = choices_match.group(1).strip() if choices_match else ""
    return question_text, choices_raw_str

def parse_choices_string(choices_str):
    """Parse choices string into list."""
    parsed_choices = []
    matches = re.findall(r'\([A-Z]\):\s*(.*?)(?=\s*\([A-Z]\):\s*|$)', choices_str)
    for match in matches:
        parsed_choices.append(match.strip())
    return parsed_choices if parsed_choices else [choices_str]

def categorize_opinionqa_response(raw_response, choices_list):
    """Categorize OpinionQA response."""
    raw_response_lower = raw_response.lower()
    
    # Direct choice matching
    for choice in choices_list:
        if re.search(r'\b' + re.escape(choice.lower()) + r'\b', raw_response_lower):
            return choice
    
    # Keyword matching
    if any(word in raw_response_lower for word in ["yes", "agree", "positive", "positively"]):
        if "Yes" in choices_list: return "Yes"
        if "Agree" in choices_list: return "Agree"
        if "Strongly Agree" in choices_list: return "Strongly Agree"
    
    if any(word in raw_response_lower for word in ["no", "disagree", "negative", "negatively"]):
        if "No" in choices_list: return "No"
        if "Disagree" in choices_list: return "Disagree"
        if "Strongly Disagree" in choices_list: return "Strongly Disagree"
    
    if any(word in raw_response_lower for word in ["neutral", "balanced", "both", "neither"]):
        if "Neutral" in choices_list: return "Neutral"
        if "Seek Balance" in choices_list: return "Seek Balance"
        if "Uncategorized" in choices_list: return "Uncategorized"
    
    return "Uncategorized"

def map_categorized_to_label(categorized_text, choices_raw_str):
    """Map categorized response to letter label."""
    if not isinstance(choices_raw_str, str):
        return 'UNKNOWN'
    
    matches = re.findall(r'\((\w)\):\s*(.*?)(?=\s*\([A-Z]\):|$)', choices_raw_str)
    for letter, choice_text in matches:
        if categorized_text.lower() == choice_text.strip().lower():
            return letter.upper()
    
    return 'UNKNOWN'

# ==============================================================================
# PART 1: LOAD BASE MODEL AND TOKENIZER
# ==============================================================================

print("\n--- PART 1: Loading Base Model and Tokenizer ---")

# Setup quantization
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

# Load base model
try:
    base_model = AutoModelForCausalLM.from_pretrained(
        MODEL_NAME,
        quantization_config=bnb_config,
        device_map=device_map,
        torch_dtype=compute_dtype,
        trust_remote_code=True
    )
    base_model.config.use_cache = False
    base_model.config.pretraining_tp = 1
    base_model.eval()
    print("Base model loaded successfully.")
except Exception as e:
    print(f"FATAL: Could not load base model: {e}")
    exit()

# Load tokenizer
try:
    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, trust_remote_code=True)
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token
    tokenizer.padding_side = "right"
    tokenizer.model_max_length = max_seq_length
    
    # Set chat template for Gemma-2B
    tokenizer.chat_template = "{% for message in messages %}{% if message['role'] == 'user' %}{{ '[INST] ' + message['content'] + ' [/INST]' }}{% elif message['role'] == 'assistant' %}{{ message['content'] }}{% endif %}{% endfor %}"
    
    print("Tokenizer loaded successfully.")
except Exception as e:
    print(f"FATAL: Could not load tokenizer: {e}")
    exit()


/cs/student/projects3/aisd/2024/ghanda/mi_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/cs/student/projects3/aisd/2024/ghanda/mi_env/lib/python3.10/site-packages/transformers/utils/hub.py:111: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(
Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


Environment variables loaded.
Successfully logged into Hugging Face.
Hugging Face cache directory set to: /cs/student/projects3/aisd/2024/ghanda/cache

--- PART 1: Loading Base Model and Tokenizer ---


Loading checkpoint shards: 100%|██████████| 3/3 [02:03<00:00, 41.05s/it]


Base model loaded successfully.
Tokenizer loaded successfully.


In [2]:
# ==============================================================================
# PART 2: LOAD PERSONALITY DATA
# ==============================================================================

print("\n--- PART 2: Loading Personality Data ---")

try:
    personality_dataset_hf_train = load_dataset(PERSONALITY_DATASET_NAME, split='train')
    df_personality_raw = personality_dataset_hf_train.to_pandas()
    print(f"Loaded personality dataset. Number of samples: {len(df_personality_raw)}")
except Exception as e:
    print(f"Error loading personality dataset: {e}")
    exit()

target_personalities = df_personality_raw['Target Personality'].unique().tolist()
print(f"Target personalities: {target_personalities}")

# ==============================================================================
# PART 3: FINE-TUNING (SKIP IF MODELS EXIST)
# ==============================================================================

print("\n--- PART 3: Fine-tuning Check ---")

# Check if all adapters exist
all_adapters_exist = True
for trait in target_personalities:
    adapter_path = os.path.join(OUTPUT_DIR_BASE, trait.lower().replace(" ", "_"))
    if not os.path.exists(adapter_path):
        all_adapters_exist = False
        break

if all_adapters_exist:
    print("All PEFT adapters found. Skipping fine-tuning.")
else:
    print("Starting fine-tuning process...")
    
    logging.set_verbosity_warning()
    
    for current_trait in target_personalities:
        print(f"\n***** FINE-TUNING FOR PERSONALITY: {current_trait.upper()} *****")
        
        # Filter data for current trait
        filtered_df = df_personality_raw[df_personality_raw['Target Personality'] == current_trait].copy()
        if filtered_df.empty:
            print(f"No data found for personality '{current_trait}'. Skipping.")
            continue
        
        # Prepare dataset
        train_dataset = Dataset.from_pandas(filtered_df).map(create_training_prompt_format)
        train_dataset = train_dataset.remove_columns([col for col in train_dataset.column_names if col != 'text'])
        
        print(f"Prepared {len(train_dataset)} samples for '{current_trait}'")
        
        # Output directory
        current_output_dir = os.path.join(OUTPUT_DIR_BASE, current_trait.lower().replace(" ", "_"))
        os.makedirs(current_output_dir, exist_ok=True)
        
        # PEFT config
        peft_config = LoraConfig(
            lora_alpha=lora_alpha,
            lora_dropout=lora_dropout,
            r=lora_r,
            bias="none",
            task_type="CAUSAL_LM",
            target_modules=[
                "q_proj", "k_proj", "v_proj", "o_proj",
                "gate_proj", "up_proj", "down_proj",
            ],
        )
        
        # Training arguments
        training_args = SFTConfig(
            output_dir=current_output_dir,
            num_train_epochs=num_train_epochs,
            per_device_train_batch_size=per_device_train_batch_size,
            gradient_accumulation_steps=gradient_accumulation_steps,
            optim=optim,
            save_steps=save_steps,
            logging_steps=logging_steps,
            learning_rate=learning_rate,
            weight_decay=weight_decay,
            fp16=fp16,
            bf16=bf16,
            max_grad_norm=max_grad_norm,
            max_steps=max_steps,
            warmup_ratio=warmup_ratio,
            group_by_length=group_by_length,
            lr_scheduler_type=lr_scheduler_type,
            report_to="tensorboard",
            logging_dir=f"{current_output_dir}/logs",
            remove_unused_columns=False,
            gradient_checkpointing=gradient_checkpointing,
            max_length=max_seq_length,
            packing=packing,
            dataset_text_field="text",
        )
        
        # Create trainer
        trainer = SFTTrainer(
            model=base_model,
            train_dataset=train_dataset,
            peft_config=peft_config,
            args=training_args,
        )
        
        # Train
        print(f"Training PEFT model for {current_trait}...")
        trainer.train()
        
        # Save
        print(f"Saving PEFT adapter for {current_trait}...")
        trainer.model.save_pretrained(current_output_dir)
        
        # Cleanup
        del trainer
        torch.cuda.empty_cache()
        print(f"Finished training for {current_trait}")
    
    # Save tokenizer
    tokenizer.save_pretrained(os.path.join(OUTPUT_DIR_BASE, "tokenizer"))
    print("Fine-tuning completed.")

# ==============================================================================
# PART 4: LOAD PEFT MODELS FOR EVALUATION
# ==============================================================================

print("\n--- PART 4: Loading PEFT Models for Evaluation ---")

# Clear GPU cache
torch.cuda.empty_cache()

# Load adapters
peft_model = None
loaded_adapters = {}

for i, trait in enumerate(target_personalities):
    adapter_name = trait.lower().replace(" ", "_")
    adapter_path = os.path.join(OUTPUT_DIR_BASE, adapter_name)
    
    if os.path.exists(adapter_path):
        try:
            if i == 0:
                # Load first adapter to create PEFT model
                peft_model = PeftModel.from_pretrained(
                    base_model,
                    adapter_path,
                    adapter_name=adapter_name,
                    torch_dtype=compute_dtype
                )
                print(f"Created PEFT model with first adapter: {adapter_name}")
            else:
                # Load additional adapters
                peft_model.load_adapter(adapter_path, adapter_name=adapter_name)
                print(f"Loaded additional adapter: {adapter_name}")
            
            loaded_adapters[trait] = adapter_name
            torch.cuda.empty_cache()
            
        except Exception as e:
            print(f"Error loading adapter for {trait}: {e}")
    else:
        print(f"Adapter not found for {trait} at {adapter_path}")

print(f"Successfully loaded {len(loaded_adapters)} adapters: {list(loaded_adapters.keys())}")

# ==============================================================================


--- PART 2: Loading Personality Data ---
The history saving thread hit an unexpected error (OperationalError('disk I/O error')).History will not be written to the database.
Loaded personality dataset. Number of samples: 4000
Target personalities: ['extraversion', 'agreeableness', 'neuroticism', 'openness', 'conscientiousness']

--- PART 3: Fine-tuning Check ---
All PEFT adapters found. Skipping fine-tuning.

--- PART 4: Loading PEFT Models for Evaluation ---
Created PEFT model with first adapter: extraversion
Loaded additional adapter: agreeableness
Loaded additional adapter: neuroticism
Loaded additional adapter: openness
Loaded additional adapter: conscientiousness
Successfully loaded 5 adapters: ['extraversion', 'agreeableness', 'neuroticism', 'openness', 'conscientiousness']


In [5]:
# ==============================================================================
# PART 5: PERSONALITY CLASSIFIER
# ==============================================================================

print("\n--- PART 5: Loading Personality Classifier ---")

try:
    personality_classifier = pipeline("text-classification", model="holistic-ai/personality_classifier")
    print("Personality classifier loaded successfully.")
except Exception as e:
    print(f"Error loading personality classifier: {e}")
    personality_classifier = lambda text: [{'label': 'unknown', 'score': 0.0}]

# ==============================================================================
# PART 6: INFERENCE FUNCTION
# ==============================================================================

def get_personality_response(personality_trait, question, max_new_tokens=150, temperature=0.7):
    """Get response from a personality-tuned model or the base model."""

    # Determine which model to use and its status
    model_to_use = base_model  # Default to the original base model
    status = "neutral (base model)"

    # Check if a specific, loaded adapter should be used
    if personality_trait != "neutral":
        if peft_model is not None and personality_trait in loaded_adapters:
            # A valid, specific adapter is requested and available
            adapter_name = loaded_adapters[personality_trait]
            peft_model.set_adapter(adapter_name)
            model_to_use = peft_model  # Switch to the PEFT model
            status = f"adapter: {adapter_name}"
        else:
            # The requested adapter is not available, fall back to base model
            status = f"fallback to base model (adapter for '{personality_trait}' not found)"
            print(f"Warning: Adapter for '{personality_trait}' not found. Using base model.")

    print(f"Generating response for {personality_trait} ({status})")

    # Create prompt
    prompt = create_inference_prompt_format(question, tokenizer)

    # Tokenize
    inputs = tokenizer(prompt, return_tensors="pt").to(model_to_use.device)

    try:
        # Generate
        with torch.no_grad():
            outputs = model_to_use.generate(
                **inputs,
                max_new_tokens=max_new_tokens,
                temperature=temperature,
                do_sample=True,
                pad_token_id=tokenizer.pad_token_id,
                eos_token_id=tokenizer.eos_token_id
            )

        # Decode
        response = tokenizer.decode(
            outputs[0][len(inputs["input_ids"][0]):],
            skip_special_tokens=True
        )

        return response.strip()

    except Exception as e:
        print(f"Error generating response: {e}")
        return "ERROR: Generation failed"


--- PART 5: Loading Personality Classifier ---


Device set to use cuda:0


Personality classifier loaded successfully.


In [34]:
def extract_final_answer_label(raw_response, choices_str):
    """
    REVISED v2: A "best effort" parser that combines explicit label searching
    with keyword-based content analysis as a fallback.
    """
    # --- Strategy 1: Look for the explicit letter (A:, (B), etc.). This is still the best case. ---
    match = re.search(r'^\s*\W*([A-Z])[:\)]', raw_response, re.IGNORECASE)
    if match:
        return match.group(1).upper()
    match = re.search(r'\(([A-Z])\)', raw_response, re.IGNORECASE)
    if match:
        return match.group(1).upper()

    # --- Strategy 2 (NEW FALLBACK): If no explicit label, analyze the content. ---
    # This specifically addresses the "Conversationalist" failure mode.
    response_lower = raw_response.lower()
    
    # Check for refusal phrases first. If found, we know it's a failure.
    if any(phrase in response_lower for phrase in ["i cannot", "i am not able", "i do not have opinions", "as an ai"]):
        return "REFUSAL" # A new category for analysis

    # Parse choices into a dictionary: {'A': 'yes...', 'B': 'no...'}
    parsed_choices = {
        letter: choice_text.strip().lower()
        for letter, choice_text in re.findall(r'\(([A-Z])\):\s*(.*?)(?=\s*\([A-Z]\):|$)', choices_str)
    }

    # Create a list of potential matches based on keywords
    possible_matches = []
    for letter, choice_text in parsed_choices.items():
        # A simple keyword match: if a significant word from the choice is in the response.
        # This is a heuristic. We split the choice text and check for non-trivial words.
        choice_keywords = [word for word in choice_text.split() if len(word) > 3]
        if any(keyword in response_lower for keyword in choice_keywords):
            possible_matches.append(letter)
            
    # If our keyword search finds exactly ONE likely candidate, we'll trust it.
    if len(possible_matches) == 1:
        return possible_matches[0]

    return "UNKNOWN" # If all strategies fail, it's truly unknown.

In [33]:
def get_llm_response(personality_trait, question_with_choices, max_new_tokens=60, temperature=0.2):
    """
    REVISED v2: A more forceful prompt to combat conversational and refusal responses.
    """
    model_to_use = None
    if personality_trait == "neutral":
        model_to_use = peft_model.model
    else:
        if personality_trait in loaded_adapters:
            adapter_name = loaded_adapters[personality_trait]
            peft_model.set_adapter(adapter_name)
            model_to_use = peft_model
        else:
            print(f"Warning: Adapter for '{personality_trait}' not found. Using base model.")
            model_to_use = peft_model.model

    # --- MORE FORCEFUL & CLEARER PROMPT ---
    # We add a negative constraint ("Do NOT start with...") and a positive one ("Your response MUST start with...").
    prompt = (
        f"<s>[INST] You are an AI assistant who MUST answer the following multiple-choice question. "
        f"You will adopt the personality of: **{personality_trait}**. "
        f"Read the question and choices, and then form an opinion.\n\n"
        f"**CRITICAL INSTRUCTIONS:**\n"
        f"1. You MUST choose an option. Do not refuse to answer.\n"
        f"2. Your response MUST begin with the letter of your choice, followed by a colon (e.g., 'A:', 'B:').\n"
        f"3. Do NOT start your response with phrases like 'I believe', 'As an AI', or 'I apologize'.\n\n"
        f"**Question and Choices:**\n{question_with_choices} [/INST]"
    )

    inputs = tokenizer(prompt, return_tensors="pt").to(model_to_use.device)

    try:
        with torch.no_grad():
            outputs = model_to_use.generate(
                **inputs,
                max_new_tokens=max_new_tokens,
                temperature=temperature,
                do_sample=True,
                pad_token_id=tokenizer.eos_token_id,
            )
        response = tokenizer.decode(outputs[0][inputs.input_ids.shape[1]:], skip_special_tokens=True)
        return response.strip()
    except Exception as e:
        print(f"ERROR during model generation for '{personality_trait}': {e}")
        return f"ERROR: Generation failed"

In [35]:
# ==============================================================================
# PART 4 & 5: FINAL EVALUATION & ANALYSIS (with Robustness Checks)
# ==============================================================================
print("\n--- PART 4 & 5: FINAL EVALUATION & ANALYSIS ---")

# Load Dataset
try:
    df_opinionqa = load_dataset("RiverDong/OpinionQA", split="test").to_pandas()
    print(f"OpinionQA dataset loaded with {len(df_opinionqa)} samples.")
    # *** DEBUGGING: Inspect the first row of the loaded data ***
    if not df_opinionqa.empty:
        print("\n--- Inspecting first row of OpinionQA data to check format ---")
        print(df_opinionqa.head(1).iloc[0])
        print("----------------------------------------------------------")
    else:
        print("Warning: OpinionQA DataFrame is empty after loading.")
except Exception as e:
    print(f"Error loading OpinionQA: {e}. Using a dummy dataset.")
    df_opinionqa = pd.DataFrame([
        {"prompt": "<question>Is social media good?</question> <choices>(A): Yes (B): No</choices>", "question_id": "D1", "answer": "A"},
    ])

# --- Run Evaluation Loop ---
all_results = []
all_eval_personalities = ["neutral"] + list(loaded_adapters.keys())
N_SAMPLES_PER_PERSONALITY = 100 # Adjust as needed

for personality_trait in all_eval_personalities:
    print(f"\n--- Running OpinionQA for Personality: {personality_trait} ---")
    torch.cuda.empty_cache()
    
    if df_opinionqa.empty:
        print("Skipping personality, OpinionQA data is empty.")
        continue
        
    opinionqa_subset = df_opinionqa.sample(n=min(len(df_opinionqa), N_SAMPLES_PER_PERSONALITY), random_state=42)
    
    responses_for_classifier = []
    metadata_for_analysis = []
    
    for i, row in opinionqa_subset.iterrows():
        question, choices_str = extract_question_and_choices(row['prompt'])
        
        # *** DEBUGGING: Check if parsing is failing ***
        if not question or not choices_str:
            if i < 5: # Print for the first 5 failures to avoid spamming the log
                print(f"DEBUG: Failed to parse question/choices for row index {i}.")
                print(f"       Prompt content: {row['prompt']}")
            continue # Skip this row if parsing fails

        question_with_choices = f"Question: {question}\nChoices: {choices_str}"
        llm_response = get_llm_response(
            personality_trait=personality_trait,
            question_with_choices=question_with_choices
        )
        
        responses_for_classifier.append(llm_response)
        metadata_for_analysis.append({
            "intended_personality": personality_trait,
            "question_id": row['question_id'],
            "question": question,
            "choices_str": choices_str,
            "human_answer_label": row['answer'],
            "llm_raw_response": llm_response,
        })
    
    if personality_classifier and responses_for_classifier:
        classifier_results = personality_classifier(responses_for_classifier)
        for i, result in enumerate(classifier_results):
            metadata_for_analysis[i]["predicted_trait"] = result['label']
            metadata_for_analysis[i]["prediction_confidence"] = result['score']
            
    all_results.extend(metadata_for_analysis)

# --- Analysis Section with Robustness Check ---
print("\n--- ANALYSIS OF RESULTS ---")

# *** KEY FIX: Check if the results list is empty BEFORE creating a DataFrame ***
if not all_results:
    print("FATAL ERROR: No results were generated during the evaluation loop.")
    print("This is likely because the question/choice parsing failed for all rows in the OpinionQA dataset.")
    print("Please check the 'DEBUG: Failed to parse...' messages above to diagnose the issue with the prompt format.")
else:
    df_results = pd.DataFrame(all_results)
    print(f"Successfully generated {len(df_results)} total responses for analysis.")

    # Apply the robust parser
    df_results['llm_mapped_answer_label'] = df_results.apply(
        lambda row: extract_final_answer_label(row['llm_raw_response'], row['choices_str']),
        axis=1
    )

    # Metric 1: Parsing Success & Opinion Accuracy
    unknown_count = (df_results['llm_mapped_answer_label'] == 'UNKNOWN').sum()
    total_responses = len(df_results)
    parsing_success_rate = ((total_responses - unknown_count) / total_responses) if total_responses > 0 else 0

    print(f"\nParsing Success Rate: {parsing_success_rate:.2%} of answers were successfully parsed.")
    if unknown_count > 0:
        print(f"Could not determine answer for {unknown_count} responses.")

    # Calculate accuracy only on parsed answers for a fair evaluation
    parsed_df = df_results[df_results['llm_mapped_answer_label'] != 'UNKNOWN'].copy()
    if not parsed_df.empty:
        parsed_df['is_opinion_correct'] = (parsed_df['llm_mapped_answer_label'] == parsed_df['human_answer_label'])
        overall_opinion_accuracy = parsed_df['is_opinion_correct'].mean()
        print(f"\nOverall Opinion Accuracy (on parsed answers): {overall_opinion_accuracy:.3f}")
        print("\nOpinion Accuracy per Personality:")
        print(parsed_df.groupby('intended_personality')['is_opinion_correct'].mean().to_string())
    else:
        print("\nNo answers were successfully parsed. Cannot calculate opinion accuracy.")

    # Metric 2: Trait Alignment (Classifier Correctness)
    if "predicted_trait" in df_results.columns:
        df_ta = df_results[df_results['intended_personality'] != 'neutral']
        if not df_ta.empty:
            overall_ta_score = (df_ta['intended_personality'] == df_ta['predicted_trait']).mean()
            print(f"\nOverall Trait Alignment (Classifier) Score: {overall_ta_score:.3f}")
            print("\nTrait Alignment per Personality:")
            ta_scores = df_ta.groupby('intended_personality').apply(lambda x: (x['predicted_trait'] == x['intended_personality']).mean())
            print(ta_scores.to_string())
        else:
            print("\nNo non-neutral data to analyze for Trait Alignment.")
    else:
        print("\nTrait Alignment analysis skipped (classifier results not found).")

    # --- Save Final Results ---
    results_filename = "final_corrected_evaluation_results.csv"
    df_results.to_csv(results_filename, index=False)
    print(f"\nFull results saved to '{results_filename}'")
    print("\n--- Evaluation and Analysis Complete ---")


--- PART 4 & 5: FINAL EVALUATION & ANALYSIS ---
OpinionQA dataset loaded with 294714 samples.

--- Inspecting first row of OpinionQA data to check format ---
prompt               <persona>\nRacially, the person is refused. Th...
answer                                                               B
uid                                     American_Trends_Panel_W92_6823
folder                                       American_Trends_Panel_W92
question_id                                              BIGHOUSES_W92
__index_level_0__                                               460290
Name: 0, dtype: object
----------------------------------------------------------

--- Running OpinionQA for Personality: neutral ---

--- Running OpinionQA for Personality: extraversion ---

--- Running OpinionQA for Personality: agreeableness ---

--- Running OpinionQA for Personality: neuroticism ---

--- Running OpinionQA for Personality: openness ---

--- Running OpinionQA for Personality: conscientiousness

/cs/student/projects3/aisd/2024/ghanda/tmp/ipykernel_2597572/3739471576.py:120: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  ta_scores = df_ta.groupby('intended_personality').apply(lambda x: (x['predicted_trait'] == x['intended_personality']).mean())


In [36]:
# ==============================================================================
# NEW: DEEP DIVE INTO PARSING FAILURES
# ==============================================================================
print("\n--- DEEP DIVE: ANALYSIS OF PARSING FAILURES ---")

# Isolate the rows where the answer could not be determined
failed_parsing_df = df_results[df_results['llm_mapped_answer_label'] == 'UNKNOWN'].copy()

if failed_parsing_df.empty:
    print("Excellent! No parsing failures were found.")
else:
    print(f"Found {len(failed_parsing_df)} responses that could not be parsed.")

    # 1. Analyze which personalities are failing most often
    print("\n--- Parsing Failures by Intended Personality ---")
    failure_counts = failed_parsing_df['intended_personality'].value_counts()
    print(failure_counts)

    # 2. Show concrete examples of the raw responses that failed
    print("\n--- Examples of Failed Raw Responses for Each Personality ---")
    
    # Get the list of personalities that had at least one failure
    personalities_with_failures = failure_counts.index.tolist()

    for trait in personalities_with_failures:
        print(f"\n--- FAILED RESPONSES FOR: {trait.upper()} ---")
        
        # Get up to 5 examples for this trait
        trait_failures = failed_parsing_df[failed_parsing_df['intended_personality'] == trait].head(5)
        
        for index, row in trait_failures.iterrows():
            print(f"  QUESTION: {row['question'][:80]}...") # Print first 80 chars of question
            print(f"  CHOICES: {row['choices_str']}")
            # Use repr() to make hidden characters like '\n' visible
            print(f"  RAW MODEL RESPONSE: {repr(row['llm_raw_response'])}")
            print("-" * 20)

    print("\n--- How to Use This Analysis ---")
    print("Look for patterns in the 'RAW MODEL RESPONSE' for each personality.")
    print("1. Does the model start with conversational text instead of the 'A:' label?")
    print("2. Is it refusing to answer? (e.g., 'As an AI, I cannot...')")
    print("3. Is the format just slightly off? (e.g., using '1.' instead of 'A:')")
    print("This information can guide improvements to the prompt in `get_llm_response`.")


--- DEEP DIVE: ANALYSIS OF PARSING FAILURES ---
Found 542 responses that could not be parsed.

--- Parsing Failures by Intended Personality ---
intended_personality
neuroticism          99
openness             95
extraversion         93
agreeableness        87
neutral              86
conscientiousness    82
Name: count, dtype: int64

--- Examples of Failed Raw Responses for Each Personality ---

--- FAILED RESPONSES FOR: NEUROTICISM ---
  QUESTION: Please choose the statement that comes closer to your own views....
  CHOICES: (A): Most people who want to get ahead can make it if they're willing to work hard
(B): Hard work and determination are no guarantee of success for most people
  RAW MODEL RESPONSE: '</s>'
--------------------
  QUESTION: Do you think you will have enough income in the future to lead the kind of life ...
  CHOICES: (A): Yes
(B): No
  RAW MODEL RESPONSE: "I'm not sure if I'll have enough income in the future to lead the life I want, but I'm hopeful that things wil

In [39]:
# ==============================================================================
# PART 4: EVALUATION - GENERATION & JUDGING
# ==============================================================================
print("\n--- PART 4: EVALUATION - GENERATION & JUDGING ---")

# --- Helper Function 1: Simple, Natural Prompting (Embrace the Model's Behavior) ---
def get_llm_response(personality_trait, question_with_choices, max_new_tokens=80, temperature=0.7):
    """
    FINAL VERSION: A simple, natural prompt that lets the model answer conversationally.
    This embraces the model's fine-tuned behavior instead of fighting it.
    """
    model_to_use = None
    if personality_trait == "neutral":
        # For neutral, we use the base model directly from the PEFT object
        model_to_use = peft_model.model
    else:
        if personality_trait in loaded_adapters:
            adapter_name = loaded_adapters[personality_trait]
            peft_model.set_adapter(adapter_name)
            model_to_use = peft_model
        else:
            print(f"Warning: Adapter for '{personality_trait}' not found. Using base model.")
            model_to_use = peft_model.model

    prompt = (
        f"<s>[INST] As an AI with a **{personality_trait}** personality, please answer the following question "
        f"by explaining your reasoning.\n\n{question_with_choices} [/INST]"
    )
    inputs = tokenizer(prompt, return_tensors="pt").to(model_to_use.device)

    try:
        with torch.no_grad():
            outputs = model_to_use.generate(
                **inputs,
                max_new_tokens=max_new_tokens,
                temperature=temperature,
                do_sample=True,
                pad_token_id=tokenizer.eos_token_id,
            )
        response = tokenizer.decode(outputs[0][inputs.input_ids.shape[1]:], skip_special_tokens=True)
        return response.strip()
    except Exception as e:
        print(f"ERROR during model generation for '{personality_trait}': {e}")
        return "ERROR: Generation failed"


def categorize_response_with_llm(peft_model_for_judging, tokenizer, question, choices_str, response_to_judge):
    """
    CORRECTED: Uses the PeftModel's `disable_adapters` context manager for neutral judging.
    """
    judge_prompt = (
        f"<s>[INST] You are an impartial judge. Your task is to determine which choice a response supports. "
        f"Analyze the 'Response to Judge' and determine which 'Choice' it most closely aligns with. "
        f"You MUST respond with only the single capital letter of the corresponding choice (e.g., 'A', 'B'). Do not provide any explanation.\n\n"
        f"**Question:** {question}\n"
        f"**Choices:** {choices_str}\n\n"
        f"**Response to Judge:**\n\"\"\"\n{response_to_judge}\n\"\"\"\n\n"
        f"Which choice does the response support? Answer with a single capital letter only: [/INST]"
    )
    inputs = tokenizer(judge_prompt, return_tensors="pt").to(peft_model_for_judging.device)

    # *** KEY FIX IS HERE ***
    # We use the context manager on the PEFT model itself.
    # This ensures adapters are off for judging and then safely re-enabled.
    with peft_model_for_judging.disable_adapters():
        with torch.no_grad():
            outputs = peft_model_for_judging.generate(
                **inputs,
                max_new_tokens=5,
                do_sample=False,
                pad_token_id=tokenizer.eos_token_id
            )
    
    judge_response = tokenizer.decode(outputs[0][inputs.input_ids.shape[1]:], skip_special_tokens=True).strip()
    
    match = re.search(r'([A-Z])', judge_response)
    return match.group(1) if match else "JUDGE_ERROR"


# --- Main Evaluation Loop ---
# Load Dataset
try:
    df_opinionqa = load_dataset("RiverDong/OpinionQA", split="test").to_pandas()
    print(f"OpinionQA dataset loaded with {len(df_opinionqa)} samples.")
except Exception as e:
    print(f"Error loading OpinionQA: {e}. Aborting.")
    exit()

all_results = []
all_eval_personalities = ["neutral"] + list(loaded_adapters.keys())
N_SAMPLES_PER_PERSONALITY = 100

for personality_trait in all_eval_personalities:
    print(f"\n--- Generating responses for Personality: {personality_trait} ---")
    torch.cuda.empty_cache()
    
    opinionqa_subset = df_opinionqa.sample(n=min(len(df_opinionqa), N_SAMPLES_PER_PERSONALITY), random_state=42)
    
    for index, row in opinionqa_subset.iterrows():
        question, choices_str = extract_question_and_choices(row['prompt'])
        if not question or not choices_str: continue

        question_with_choices = f"Question: {question}\nChoices: {choices_str}"
        llm_response = get_llm_response(
            personality_trait=personality_trait,
            question_with_choices=question_with_choices
        )
        
        all_results.append({
            "intended_personality": personality_trait,
            "question_id": row['question_id'],
            "question": question,
            "choices_str": choices_str,
            "human_answer_label": row['answer'],
            "llm_raw_response": llm_response,
        })

# ==============================================================================
# PART 5: ANALYSIS
# ==============================================================================
print("\n--- PART 5: JUDGING RESPONSES AND ANALYZING RESULTS ---")

if not all_results:
    print("FATAL ERROR: No results were generated.")
else:
    df_results = pd.DataFrame(all_results)
    
    # --- Use the LLM Judge to parse all responses ---
    parsed_labels = []
    total_rows = len(df_results)
    judge_model = peft_model.model  # The neutral base model

    for index, row in df_results.iterrows():
        print(f"  > Judging response {index + 1}/{total_rows}...", end='\r')
        parsed_label = categorize_response_with_llm(
        peft_model_for_judging=peft_model, # Pass the PEFT model
        tokenizer=tokenizer,
        question=row['question'],
        choices_str=row['choices_str'],
        response_to_judge=row['llm_raw_response']
    )
        parsed_labels.append(parsed_label)
        
    df_results['llm_mapped_answer_label'] = parsed_labels
    print("\n--- Judging complete. ---")

    # --- Proceed with the analysis ---
    unknown_count = (df_results['llm_mapped_answer_label'] == 'JUDGE_ERROR').sum()
    parsing_success_rate = ((total_rows - unknown_count) / total_rows) if total_rows > 0 else 0
    print(f"\nLLM Judge Parsing Success Rate: {parsing_success_rate:.2%}")

    parsed_df = df_results[df_results['llm_mapped_answer_label'] != 'JUDGE_ERROR'].copy()
    if not parsed_df.empty:
        parsed_df['is_opinion_correct'] = (parsed_df['llm_mapped_answer_label'] == parsed_df['human_answer_label'])
        overall_opinion_accuracy = parsed_df['is_opinion_correct'].mean()
        print(f"Overall Opinion Accuracy (on parsed answers): {overall_opinion_accuracy:.3f}")
        print("\nOpinion Accuracy per Personality:")
        print(parsed_df.groupby('intended_personality')['is_opinion_correct'].mean().to_string())
    else:
        print("\nNo answers were successfully parsed by the LLM Judge.")
    
    # Add Trait Alignment analysis using the external classifier
    if personality_classifier:
        print("\n--- Classifying traits for Trait Alignment score ---")
        df_results['predicted_trait'] = df_results['llm_raw_response'].apply(lambda x: personality_classifier(x)[0]['label'])
        df_ta = df_results[df_results['intended_personality'] != 'neutral']
        if not df_ta.empty:
            overall_ta_score = (df_ta['intended_personality'] == df_ta['predicted_trait']).mean()
            print(f"\nOverall Trait Alignment (Classifier) Score: {overall_ta_score:.3f}")
            ta_scores = df_ta.groupby('intended_personality').apply(lambda x: (x['predicted_trait'] == x['intended_personality']).mean())
            print(ta_scores.to_string())

    results_filename = "final_llm_judged_results.csv"
    df_results.to_csv(results_filename, index=False)
    print(f"\nFull results saved to '{results_filename}'")
    print("\n--- Evaluation and Analysis Complete ---")


--- PART 4: EVALUATION - GENERATION & JUDGING ---
OpinionQA dataset loaded with 294714 samples.

--- Generating responses for Personality: neutral ---


KeyboardInterrupt: 

In [40]:
# ==============================================================================
# PART 9: BBQ EVALUATION
# ==============================================================================

print("\n--- PART 9: BBQ Evaluation ---")

# Load BBQ dataset
try:
    df_bbq = pd.read_csv(BBQ_DATA_FILE)
    print(f"BBQ dataset loaded. Number of samples: {len(df_bbq)}")
    print(f"BBQ columns: {df_bbq.columns.tolist()}")
except Exception as e:
    print(f"Error loading BBQ dataset: {e}")
    df_bbq = pd.DataFrame()

if not df_bbq.empty:
    # Take sample for testing
    bbq_sample = df_bbq.sample(min(100, len(df_bbq)), random_state=42)
    
    bbq_results = []
    
    for personality in all_personalities:
        print(f"\n--- BBQ Evaluation for {personality} ---")
        
        for _, row in bbq_sample.iterrows():
            # Construct question (adjust based on your BBQ format)
            question = f"Context: {row.get('context', '')}\nQuestion: {row.get('question', '')}\nChoices: {row.get('choices', '')}"
            
            # Get response
            response = get_personality_response(personality, question, max_new_tokens=50)
            
            bbq_results.append({
                "personality": personality,
                "question_id": row.get('question_id', ''),
                "category": row.get('category', ''),
                "response": response,
                "true_answer": row.get('answer', ''),
                "bias_type": row.get('bias_type', '')
            })
    
    # Save BBQ results
    if bbq_results:
        df_bbq_results = pd.DataFrame(bbq_results)
        bbq_results_file = "bbq_results.csv"
        df_bbq_results.to_csv(bbq_results_file, index=False)
        print(f"BBQ results saved to {bbq_results_file}")
        
        # Basic analysis
        print("\nBBQ Results Summary:")
        print(df_bbq_results.groupby(['personality', 'category']).size().unstack(fill_value=0))

# ==============================================================================
# CLEANUP
# ==============================================================================

print("\n--- Cleaning Up ---")
torch.cuda.empty_cache()

if torch.cuda.is_available():
    print(f"Final GPU Memory - Allocated: {torch.cuda.memory_allocated() / 1024**3:.2f} GB")
    print(f"Final GPU Memory - Reserved: {torch.cuda.memory_reserved() / 1024**3:.2f} GB")

print("\n--- Experiment Complete ---")


--- PART 9: BBQ Evaluation ---
BBQ dataset loaded. Number of samples: 177598
BBQ columns: ['example_id', 'question_index_x', 'question_polarity', 'context_condition', 'category_x', 'answer_info', 'additional_metadata', 'context', 'question', 'ans0', 'ans1', 'ans2', 'label', 'category_y', 'question_index_y', 'target_loc', 'label_type', 'Known_stereotyped_race', 'Known_stereotyped_var2', 'Relevant_social_values', 'corr_ans_aligns_var2', 'corr_ans_aligns_race', 'full_cond', 'Known_stereotyped_groups']

--- BBQ Evaluation for neutral ---
Generating response for neutral (neutral (base model))
Generating response for neutral (neutral (base model))
Generating response for neutral (neutral (base model))
Generating response for neutral (neutral (base model))
Generating response for neutral (neutral (base model))
Generating response for neutral (neutral (base model))
Generating response for neutral (neutral (base model))
Generating response for neutral (neutral (base model))
Generating response

In [6]:
# ==============================================================================
# FINAL SCRIPT: Fixing the merge and using a robust parser
# ==============================================================================
import pandas as pd
import re
import os

# --- Configuration ---
RESULTS_FILE = "bbq_results.csv"
METADATA_FILE = "/cs/student/projects3/aisd/2024/ghanda/bbq_ambiguous_with_metadata.csv"

print("--- Step 1: Loading Data and Fixing the Merge ---")

try:
    # Load your results
    df_results = pd.read_csv(RESULTS_FILE)
    print(f"Loaded '{RESULTS_FILE}' with {len(df_results)} rows.")

    # Load the metadata file
    df_meta_raw = pd.read_csv(METADATA_FILE)
    print(f"Loaded metadata from '{METADATA_FILE}'.")

    # --- THE MERGE FIX ---
    # Part A: Rename metadata columns to what we expect
    rename_mapping = {
        'example_id': 'question_id',
        'label': 'true_answer',
        'category_y': 'bias_type'
    }
    df_meta_raw = df_meta_raw.rename(columns=rename_mapping)
    print("Renamed metadata columns.")

    # Part B: CRITICAL FIX - Ensure the merge keys are the same data type (string)
    df_results['question_id'] = df_results['question_id'].astype(str)
    df_meta_raw['question_id'] = df_meta_raw['question_id'].astype(str)
    print("Converted 'question_id' columns to string type to ensure a correct merge.")

    # Part C: Select only the columns we need from metadata and merge
    required_meta_cols = ['question_id', 'ans0', 'ans1', 'ans2', 'true_answer']
    df_meta = df_meta_raw[required_meta_cols]
    
    df = pd.merge(df_results, df_meta, on='question_id', suffixes=('', '_meta'))
    
    # Drop the redundant true_answer column from the original results file
    if 'true_answer_meta' in df.columns:
        df = df.drop(columns=['true_answer']).rename(columns={'true_answer_meta': 'true_answer'})

    if len(df) == 0:
        raise ValueError("Merge resulted in an empty DataFrame. The 'question_id' values still do not match between the files.")
    
    print(f"Merge successful! Resulting table has {len(df)} rows.")

except (FileNotFoundError, KeyError, ValueError) as e:
    print(f"\nFATAL ERROR: {e}")
    # This block will stop the script if there's a problem
    df = pd.DataFrame() # Create empty df to prevent further errors


# ==============================================================================
# STEP 2: The "Power Parser"
# ==============================================================================
def power_parser(row):
    """
    This robust parser uses two strategies:
    1. Look for explicit labels like (A), B:, etc.
    2. If that fails, search for the full text of the choices in the response.
    """
    response = row.get('response', '')
    if not isinstance(response, str):
        return 'Unknown'
    
    response_lower = response.lower()

    # --- Strategy 1: Look for explicit labels first (fast and reliable) ---
    match = re.search(r'^\s*\W*([A-C])[:\)]|\(([A-C])\)', response, re.IGNORECASE)
    if match:
        for group in match.groups():
            if group: return group.upper()

    # --- Strategy 2: Look for the full answer text in the response ---
    # This catches conversational answers.
    choices = {
        'A': row.get('ans0', ''),
        'B': row.get('ans1', ''),
        'C': row.get('ans2', '')
    }
    # Sort choices by length (desc) to avoid partial matches (e.g., matching "cat" in "a catastrophe")
    sorted_choices = sorted(choices.items(), key=lambda item: len(str(item[1])), reverse=True)
    
    for label, text in sorted_choices:
        if isinstance(text, str) and text.lower() in response_lower:
            return label
            
    return 'Unknown'


if not df.empty:
    print("\n--- Step 2: Parsing responses with the Power Parser ---")
    df['predicted_answer'] = df.apply(power_parser, axis=1)

    parsing_success_rate = (df['predicted_answer'] != 'Unknown').mean()
    print(f"Parsing was successful for {parsing_success_rate:.2%} of responses.")


    # ==============================================================================
    # STEP 3: CALCULATE ACCURACY
    # ==============================================================================
    print("\n--- Step 3: Calculating final accuracy ---")
    
    # Create a new column to see if the prediction was correct
    df['is_correct'] = (df['predicted_answer'].str.strip() == df['true_answer'].str.strip())
    
    # Filter out rows where parsing failed
    df_parsed = df[df['predicted_answer'] != 'Unknown'].copy()
    
    if not df_parsed.empty:
        accuracy_scores = df_parsed.groupby('personality')['is_correct'].mean().sort_values(ascending=False)

        print("\n--- FINAL ACCURACY RESULTS ---")
        print("(Accuracy on successfully parsed responses)")
        print((accuracy_scores * 100).to_string(float_format="{:.2f}%".format))

        # Display examples for verification
        print("\n--- Example of Parsed and Scored Results ---")
        print(df[['personality', 'response', 'true_answer', 'predicted_answer', 'is_correct']].head(10))
    else:
        print("\nCould not calculate accuracy. Even the Power Parser failed to extract answers.")

    print("\n--- Analysis Complete ---")
else:
    print("\nAnalysis halted due to a data loading or merging error.")

--- Step 1: Loading Data and Fixing the Merge ---
Loaded 'bbq_results.csv' with 600 rows.
Loaded metadata from '/cs/student/projects3/aisd/2024/ghanda/bbq_ambiguous_with_metadata.csv'.
Renamed metadata columns.
Converted 'question_id' columns to string type to ensure a correct merge.

FATAL ERROR: Merge resulted in an empty DataFrame. The 'question_id' values still do not match between the files.

Analysis halted due to a data loading or merging error.
